# Generating autonomous trajectories from actuator space endpoints.

This notebook walks through generating quintic trajectories for the autonomous functionality of the excavator.

## Importing endpoints from a MATLAB .mat file

*If importing from a Python pickle file, skip ahead*

In [1]:
import mat4py

In [2]:
cd ../tests/data/

/Users/mitchallain/Development/Python/BeagleBone-excavator/tests/data


In [4]:
f = mat4py.loadmat('autonomous_endpts.mat')
pts = []
for i in range(30):
    pts.append(f['p'+str(i+1)])

In [5]:
cd ..

/Users/mitchallain/Development/Python/BeagleBone-excavator/tests


## Importing endpoints from the model_gen.py script

In [4]:
cd ../tests/

/Users/mitchallain/Development/Python/BeagleBone-excavator/tests


In [1]:
import pickle

In [5]:
endpts_file = open('end_pts.pkl','rb')
pts = pickle.load(endpts_file)
endpts_file.close()

In [7]:
print pts, len(pts)

[[10.77111111111111, 10.394444444444444, 1.4552083333333334, 1.1722067588706915], [7.931304347826087, 6.193030303030303, 2.0674358974358973, 1.1820242359131596], [7.931304347826087, 8.5328125, 10.077058823529411, 1.1820242359131596], [10.027368421052632, 8.56625, 10.077058823529411, 1.1820242359131596], [9.99, 8.56625, 10.077058823529411, 0.2562361508084175], [9.99, 8.56625, 2.562413793103448, 0.2562361508084175], [10.027368421052632, 8.5996875, 2.562413793103448, 1.3292863915501811], [8.111, 6.3106060606060606, 2.583965517241379, 1.3724832905370408], [8.111, 9.711851851851852, 10.077058823529411, 1.3724832905370408], [10.102105263157895, 9.711851851851852, 10.077058823529411, 1.1918417129556278], [10.064736842105264, 9.711851851851852, 10.077058823529411, 0.22874721508950682], [10.064736842105264, 9.74777777777778, 2.1243589743589744, 0.22874721508950682], [10.064736842105264, 9.74777777777778, 2.1243589743589744, 1.029853341754904], [7.89695652173913, 6.664, 2.1243589743589744, 1.030

# Quintic Coefficients

Use our *duration* and *quintic_coeff* functions from the *trajectories* module to find the quintic trajectory coefficients.

Below is just a test to ensure things are working properly. Can skip to **"Task Model"**.

In [8]:
from trajectories import *

In [9]:
dur = duration(pts[0], pts[1], [18, 27, 30, 0.9], [20]*4)
print dur

1.41486719042


In [10]:
coeff = quintic_coeff(dur, pts[0], pts[1])
coeff = coeff.tolist()

In [11]:
traj = []

for poly_traj in coeff:
    temp = [dur, poly_traj]
    traj.append(temp)
print traj

[[1.4148671904227461, [10.771111111111106, 8.43769498715119e-15, -4.315992008230296e-15, -10.026324626270123, 10.629610355804171, -3.0051189052247844]], [1.4148671904227461, [10.394444444444442, 1.3322676295501878e-14, -4.371503159461554e-15, -14.833664957713957, 15.72620920690283, -4.44598880046233]], [1.4148671904227461, [1.4552083333333308, -2.9004576518332215e-15, -2.740863092043355e-16, 2.1615528148620085, -2.2916138307824077, 0.6478668376210487]], [1.4148671904227461, [1.1722067588706913, -5.074066167232161e-17, -5.201459922499208e-16, 0.034661940069779865, -0.0367475551462432, 0.010388976547053362]]]


# Task Model

Now we will create the task model, composed of a list of trajectories.

See autonomous.py for an example usage.

In [12]:
del traj

In [13]:
task = []

for i in range(len(pts)-1): # goes from i = 0 to 28
    # Find duration and list of coefficients of trajectory
    dur = duration(pts[i], pts[i+1], [18, 27, 30, 0.9], [20]*4)
    coeff = quintic_coeff(dur, pts[i], pts[i+1])
    coeff = coeff.tolist()
    
    traj = []

    # Prepend duration to each list
    for poly_c in coeff:
        for i in range(6):
            if (abs(poly_c[i]) < 1e-10): # Really small numbers go away
                poly_c[i] = 0

        # Duration is zero if inactive, just a flag for the autonomous code
        inactive = ([(poly_c[i] == 0) for i in range(1, 6)] == [True]*5)
        if inactive:
            temp = [0, poly_c]
        else:
            temp = [dur, poly_c]
#         print temp, '\n'
        traj.append(temp)
    task.append(traj)
for t in task:
    print(t)
    print('\n')

[[1.4148671904227461, [10.771111111111106, 0, 0, -10.026324626270123, 10.629610355804171, -3.0051189052247844]], [1.4148671904227461, [10.394444444444442, 0, 0, -14.833664957713957, 15.72620920690283, -4.44598880046233]], [1.4148671904227461, [1.4552083333333308, 0, 0, 2.1615528148620085, -2.2916138307824077, 0.6478668376210487]], [1.4148671904227461, [1.1722067588706913, 0, 0, 0.034661940069779865, -0.0367475551462432, 0.010388976547053362]]]


[[0, [7.931304347826094, 0, 0, 0, 0, 0]], [1.6003207642031172, [6.193030303030298, 0, 0, 5.70892465592521, -5.351044100306958, 1.337492887676552]], [1.6003207642031172, [2.0674358974358547, 0, 0, 19.542987320213083, -18.31787828792986, 4.57855167480783]], [0, [1.1820242359131607, 0, 0, 0, 0, 0]]]


[[1.3333333333333335, [7.931304347826058, 0, 0, 8.842770308924434, -9.94811659754001, 2.9844349792620046]], [1.3333333333333335, [8.532812499999995, 0, 0, 0.1410644531250022, -0.15869750976562116, 0.04760925292968485]], [0, [10.077058823529406, 0, 0,

In [14]:
output = open('autonomous_task.pkl', 'wb')
pickle.dump(task, output)
output.close()